In [1]:
import json
import pandas as pd
import altair as alt
import os
import numpy as np

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [2]:
@alt.theme.register('theme_name', enable=True)
def set_default_colorscheme()  -> alt.theme.ThemeConfig:
    return {
        "config": {
            "range": {
                "category": ["#1f77b4", "#ff7f0e", "#2ca02c", "#e45756", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#efb118", "#17becf", "#e41a1c"]
                #"category": ["#4269d0", "#efb118", "#ff725c", "#6cc5b0", "#3ca951", "#ff8ab7", "#a463f2", "#97bbf5", "#9c6b4e", "#9498a0", "#9498a0"]
            },
            "axis": {
                "titleFontSize": 34,
                "labelFontSize": 32, 
                "limit": 0,
            },
            
            "legend": {
                "titleFontSize": 32,
                "labelFontSize": 30,
                "labelLimit": 0,
                "titleLimit": 0,
            },
            "title": {
                "fontSize": 34,
                "limit": 0,
            },
            "text": {
                "fontSize": 32, 
                "limit": 0,
                #"fontWeight": "bold",
            },
        }
    }

In [3]:
perf_logs = [json.load(open('performance_test_log_file.json'))]

In [4]:
query_categories = {
    'samples_with_specific_elements': 'Mat. libraries', 
    'samples_with_specific_elements_exclusive': 'Mat. libraries', 
    'samples_with_substrates': 'Mat. libraries', 
    'samples_not_composed_of_elements': 'Mat. libraries', 
    'samples_and_authors': 'Mat. libraries', 
    
    'query_user_details': 'Users', 
    'samples_per_user': 'Users', 
    
    'query_based_on_composition_values': 'Compositions', 
    'composition_count_per_MA': 'Compositions', 

    'measurement_to_handovers': 'Handovers', 
    'handovers_with_specific_measurement_type': 'Handovers', 

    'get_edx_csv_measurements': 'Measurements', 
    'measurements_and_authors': 'Measurements'
}

In [5]:
materialization_times = []
conversion_times = []
per_mapping_times = []

cpu_usage_trace = []
per_mapping_memory_usage = []

n_triples = []
postprocessing_times = []
postprocessing_times_total = []

cpu_usage_trace_postprocessing = []
memory_usage_trace_postprocessing = []

query_benchmark_results = []

for perf_log in perf_logs:
    for test in perf_log:
        # Skip debugging tests
        if test["config"]["num_main_samples"] not in [704, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]:
            continue
        
        max_measurements_in_main_samples = test["config"]["max_measurements_in_main_samples"]
        if max_measurements_in_main_samples == 4:
            multiplier = "Original activity"
        elif max_measurements_in_main_samples == 5:
            multiplier = "+25%"
        elif max_measurements_in_main_samples == 6:
            multiplier = "+50%"
        elif max_measurements_in_main_samples == 7:
            multiplier = "+75%"
        elif max_measurements_in_main_samples == 8:
            multiplier = "+100%"
        else:
            continue # Skip debugging tests
    
        n_triples.append({"n_samples": test["config"]["num_main_samples"], 
                          "multiplier": multiplier, 
                          "n_triples": test["n_triples_generated"]})
    
        # Query benchmark
        for query, (time_sql, time_sparql) in test["query_benchmark_results"].items():
            query_benchmark_results.append({"n_samples": test["config"]["num_main_samples"], 
                                            "multiplier": multiplier, 
                                            "query": query,
                                            "category": query_categories[query],
                                            "time": time_sql,
                                            "type": "sql"})
    
            query_benchmark_results.append({"n_samples": test["config"]["num_main_samples"], 
                                            "multiplier": multiplier, 
                                            "query": query,
                                            "category": query_categories[query],
                                            "time": time_sparql,
                                            "type": "sparql"})
        
        # Materialization times
        for base_mapping_path, mapping_logs in test["mappings_performance_log"]["per_mapping_times"].items():
            category = os.path.basename(os.path.dirname(base_mapping_path)).replace("_", " ").capitalize()

            for mapping_path, mapping_log in mapping_logs.items():
                mapping_name = os.path.splitext(os.path.basename(mapping_path))[0].replace("_templated", "").replace("_", " ")
                
                per_mapping_times.append({"mapping_name": mapping_name,
                                          "category": category,
                                          "n_samples": test["config"]["num_main_samples"],
                                          "mat_time": mapping_log["rml_materialization"],
                                          "multiplier": multiplier,
                                          "n_run": test["config"]["n_run"]})
        
        # Postprocessing times
        postprocessing_time_sum = 0
        for postprocessing_step, postprocessing_time in test["postprocessing_performance_log"].items():    
            postprocessing_time_sum += postprocessing_time
            postprocessing_times.append({"step": postprocessing_step,
                                         "n_samples": test["config"]["num_main_samples"],
                                         "multiplier": multiplier, 
                                         "time": postprocessing_time})
        
        postprocessing_times_total.append({"n_samples": test["config"]["num_main_samples"],
                                         "multiplier": multiplier, 
                                         "time": postprocessing_time_sum,
                                         "n_run": test["config"]["n_run"]})
    
        # Materialization resource usage
        for i, (cpu_usage, memory_usage) in enumerate(test["resource_usage_mappings"]):
            cpu_usage_trace.append({ 
                "time": i, # Each tick is a second
                "cpu_usage": cpu_usage,
                "n_samples": test["config"]["num_main_samples"],
                "multiplier": multiplier,
                "n_run": test["config"]["n_run"]
            })
    
            for (mapping_path, memory_usage) in memory_usage["used_mem_per_file"].items():
                mapping_name = os.path.splitext(os.path.basename(mapping_path))[0].replace("_templated", "").replace("_", " ")
                category = os.path.basename(os.path.dirname(mapping_path)).replace("_", " ").capitalize()
            
                per_mapping_memory_usage.append({ 
                    "time": i, # Each tick is a second
                    "mapping_name": mapping_name,
                    "category": category,
                    "memory_usage": memory_usage,
                    "n_samples": test["config"]["num_main_samples"],
                    "multiplier": multiplier,
                    "n_run": test["config"]["n_run"]
                })
    
        # Postprocessing resource usage
        for i, (cpu_usage, memory_usage) in enumerate(test["resource_usage_postprocessing"]):
            cpu_usage_trace_postprocessing.append({ 
                "time": i, # Each tick is a second
                "cpu_usage": cpu_usage,
                "n_samples": test["config"]["num_main_samples"],
                "multiplier": multiplier,
                "n_run": test["config"]["n_run"]
            })
    
            memory_usage_trace_postprocessing.append({ 
                "time": i, # Each tick is a second
                "memory_usage": memory_usage,
                "n_samples": test["config"]["num_main_samples"],
                "multiplier": multiplier,
                "n_run": test["config"]["n_run"]
            })
            
        yarrrml_to_rml_conversion_real_time = test["mappings_performance_log"]["yarrrml_to_rml_conversion_real_time"]
        materialization_real_time = test["mappings_performance_log"]["materialization_real_time"]
        
        materialization_times.append({"n_samples": test["config"]["num_main_samples"],
                                      "real_time": materialization_real_time,
                                      "multiplier": multiplier,
                                      "n_run": test["config"]["n_run"]})

        conversion_times.append({"n_samples": test["config"]["num_main_samples"],
                                 "yarrrml_to_rml_conversion_real_time": yarrrml_to_rml_conversion_real_time,
                                 "multiplier": multiplier,
                                 "n_run": test["config"]["n_run"]})

materialization_times = pd.DataFrame(materialization_times)
conversion_times = pd.DataFrame(conversion_times)
per_mapping_times = pd.DataFrame(per_mapping_times)
cpu_usage_trace = pd.DataFrame(cpu_usage_trace)
per_mapping_memory_usage = pd.DataFrame(per_mapping_memory_usage)
n_triples = pd.DataFrame(n_triples)
postprocessing_times = pd.DataFrame(postprocessing_times)
postprocessing_times_total = pd.DataFrame(postprocessing_times_total)
cpu_usage_trace_postprocessing = pd.DataFrame(cpu_usage_trace_postprocessing)
memory_usage_trace_postprocessing = pd.DataFrame(memory_usage_trace_postprocessing)
query_benchmark_results = pd.DataFrame(query_benchmark_results)

# Simplify the categories for plots:
#    1) Reqs.for synthesis and ideas mappings are separated as they will be expanded in the future, but act as measurements right now
#    2) Handovers and activities are separated for readability, but both belong to the handover workflows construction
#    3) Users are also in their own category, but their mappings amount to nothing - we can consider them as part of objects
per_mapping_times.loc[per_mapping_times['category'] == 'Requests for synthesis', 'category'] = 'Measurements'
per_mapping_times.loc[per_mapping_times['category'] == 'Ideas', 'category'] = 'Measurements'
per_mapping_times.loc[per_mapping_times['category'] == 'Users', 'category'] = 'Objects'
per_mapping_times.loc[per_mapping_times['category'] == 'Samples', 'category'] = 'Objects'
per_mapping_times.loc[per_mapping_times['category'] == 'Handovers', 'category'] = 'Handovers'
per_mapping_times.loc[per_mapping_times['category'] == 'Activities', 'category'] = 'Handovers'

# Number of triples per number of samples

In [6]:
base = alt.Chart(n_triples).encode(
    x=alt.X('n_samples', title="Number of materials libraries"),
    y=alt.Y('mean(n_triples)', title="RDF triples generated", axis=alt.Axis(tickCount=10)),
    color=alt.Color(
        'multiplier',
        sort=['Original activity', '+25%', '+50%', '+75%', '+100%'],
        legend=alt.Legend(title="Increased activity modifier", orient="top")
    ),
    shape=alt.Shape('multiplier', legend=None),
    strokeDash=alt.StrokeDash('multiplier', legend=None)
)

line = base.mark_line(size=3)  # thicker line
points = base.mark_point(size=100)  # add points

text = base.mark_text(
    align='center',
    baseline='middle',
    dy=-20
).encode(
    text=alt.Text('mean(n_triples):Q', format='.3s')#,.0f')
)


error_bands = alt.Chart(n_triples).mark_errorband(extent='stdev').encode(
    x='n_samples:Q',
    y=alt.Y('n_triples:Q', title=''),# title='Average number of triples'),
    color=alt.Color(
        'multiplier:N',
        sort=['Original activity', '+25%', '+50%', '+75%', '+100%'],
        legend=None
    )
)


n_triples_plot = (line + points + text + error_bands).properties(
    width=1920, # For the paper: 800
    height=1080 # For the paper: 600
)

n_triples_plot.save('n_triples.pdf')
n_triples_plot

alt.LayerChart(...)

# Postprocessing time

In [7]:
chart = alt.Chart(postprocessing_times_total).mark_line().encode(
    x=alt.X('n_samples:Q', title="Number of materials libraries"),
    # Note: Avg across runs, sum over all postprocessing steps for each x tick and multiplier
    y=alt.Y('mean(time):Q', title="Postprocessing time (seconds)"),
    color=alt.Color(
        'multiplier',
        sort=['Original activity', '+25%', '+50%', '+75%', '+100%'], 
        legend=alt.Legend(
            title="Increased activity modifier",
            orient="top"
        )
    ),
    shape=alt.Shape('multiplier', legend=None),
    strokeDash=alt.StrokeDash('multiplier', legend=None)
).properties(
    width=1920, # For the paper: 800
    height=1080  # For the paper: 600
)

line = chart.mark_line(size=3) 
points = chart.mark_point(size=100) 

text = chart.mark_text(
    align='center',
    baseline='middle',
    dx=0,
    dy=-20
).encode(
    text=alt.Text('mean(time):Q', format=".1f")
)

error_bands = alt.Chart(postprocessing_times_total).mark_errorband(extent='stdev').encode(
    x='n_samples:Q',
    y=alt.Y('mean(time):Q', title='Postprocessing time (seconds)'),
    color=alt.Color(
        'multiplier:N',
        sort=['Original activity', '+25%', '+50%', '+75%', '+100%'],
        legend=None
    )
)

postprocessing_plot = (chart + line + points + text + error_bands)


postprocessing_plot.save('total_postprocessing_time_per_n_samples.pdf')

# Postprocessing CPU usage

In [8]:
charts = [  
    alt.Chart(cpu_usage_trace_postprocessing.query("multiplier == @m")).mark_line().encode(
        x=alt.X('time:N', title="Elapsed time (seconds)", axis=alt.Axis(labelAngle=0)),
        y=alt.Y('mean(cpu_usage):Q', title="Total CPU usage (Avg. across repetitions)"),
        color=alt.Color(
            'n_samples:N',
            legend=alt.Legend(
                title="Number of materials libraries",
                orient="top"
            )
        ),
        shape=alt.Shape('n_samples:N', legend=None),
        strokeDash=alt.StrokeDash('n_samples:N', legend=None)
    ).properties(title=f"Multiplier: {m}", width=1920, height=1080)
    for m in ['Original activity', '+25%', '+50%', '+75%', '+100%']
]

top = alt.hconcat(*charts[:3], spacing=20)
bottom = alt.hconcat(
    alt.Chart().mark_text(text='').properties(width=125), 
    *charts[3:],
    alt.Chart().mark_text(text='').properties(width=125)   
)


title = alt.TitleParams(
    text="Postprocessing CPU usage (% of total use over 24c/48t) (Avg. across all tests)",
    offset=100,
    anchor='middle',
    fontSize=64
)
chart = alt.vconcat(top, bottom, spacing=40, title=title)


chart.save('postprocessing_cpu_usage.pdf')

# Postprocessing memory usage

In [9]:
memory_usage_trace_postprocessing['memory_usage_gib'] = memory_usage_trace_postprocessing['memory_usage'] / (1024**3)

In [10]:
memory_usage_trace_postprocessing["memory_usage_gib"]

0       181.396343
1       181.505840
2       181.613506
3       181.614239
4       181.614239
           ...    
4694    184.112045
4695    184.114243
4696    184.114975
4697    184.114975
4698    184.114975
Name: memory_usage_gib, Length: 4699, dtype: float64

In [11]:
charts = [  
    alt.Chart(memory_usage_trace_postprocessing.query("multiplier == @m")).mark_line().encode(
        x=alt.X('time:Q', title="Elapsed time (seconds)", axis=alt.Axis(labelAngle=0, tickCount=10)),
        y=alt.Y('mean(memory_usage_gib)', 
                title="Memory usage (GiB) (Avg. across repetitions)", 
                axis=alt.Axis(format=".1f", tickCount=5),
                scale=alt.Scale(zero=False)),
        color=alt.Color(
            'n_samples:N',
            legend=alt.Legend(
                title="Number of materials libraries",
                orient="top"
            )
        ),
        shape=alt.Shape('n_samples:N', legend=None),
        strokeDash=alt.StrokeDash('n_samples:N', legend=None)
    ).properties(title=f"Multiplier: {m}", width=1920, height=1080)
    for m in ['Original activity', '+25%', '+50%', '+75%', '+100%']
]

top = alt.hconcat(*charts[:3], spacing=20)
bottom = alt.hconcat(
    alt.Chart().mark_text(text='').properties(width=125),  
    *charts[3:],
    alt.Chart().mark_text(text='').properties(width=125)   
)

title = alt.TitleParams(
    text="Postprocessing memory usage (starting values indicate Virtuoso's initial reserved memory) (Avg. across all tests)",
    offset=100,
    anchor='middle',
    fontSize=64
)

chart = alt.vconcat(top, bottom, spacing=40, title=title)

chart.save('postprocessing_memory_usage.pdf')

# Total materialization time per number of samples

In [12]:
materialization_times

,n_samples,real_time,multiplier,n_run
0,704,55.351614,+25%,0
1,1000,57.278403,+25%,0
2,2000,127.720173,+25%,0
3,3000,176.900779,+25%,0
4,4000,145.072490,+25%,0
...,...,...,...,...
160,6000,1694.376703,+100%,2
161,7000,2203.878119,+100%,2
162,8000,2200.648775,+100%,2
163,9000,2524.543448,+100%,2


In [13]:
chart = alt.Chart(materialization_times).mark_line().encode(
    x=alt.X('n_samples', title="Number of materials libraries"),
    y=alt.Y('mean(real_time)', title="Materialization time (seconds)"),
    color=alt.Color(
        'multiplier',
        sort=['Original activity', '+25%', '+50%', '+75%', '+100%'], 
        legend=alt.Legend(
            title="Multiplier value",
            orient="top"
        )
    ),
    shape=alt.Shape('multiplier', legend=None),
    strokeDash=alt.StrokeDash('multiplier', legend=None)
).properties(
    width=1920, # For the paper: 800
    height=1080 # For the paper: 600
)

line = chart.mark_line(size=3)
points = chart.mark_point(size=100)

text_1 = chart.mark_text(
    align='center',
    baseline='middle',
    dx=0,
    dy=-20
).encode(
    text=alt.condition(
        alt.datum.multiplier != "No multiplier",
        alt.Text('mean(real_time):Q', format=",.1f"),
        alt.value('')
    )
)

text_2 = chart.mark_text(
    align='center',
    baseline='middle',
    dx=0,
    dy=-20
).encode(
    text=alt.condition(
        alt.datum.multiplier == "No multiplier",
        alt.Text('mean(real_time):Q', format=",.1f"),
        alt.value('')
    )
)

error_bands = alt.Chart(materialization_times).mark_errorband(extent='stdev').encode(
    x='n_samples:Q',
    y=alt.Y('real_time:Q', title='Materialization time (seconds)'),
    color=alt.Color(
        'multiplier:N',
        sort=['Original activity', '+25%', '+50%', '+75%', '+100%'],
        legend=None
    )
)

chart = (chart + line + points + text_1 + text_2 + error_bands)

chart.save('materialization_time_per_n_samples.pdf')

# Avg. YARRRML -> RML conversion time

This is just to see that the overhead of the Docker containers for the YARRRML conversion is hurting us. Since this can be done
in an offline manner after modifying the mappings, we exclude it and use the maximum materialization time instead.

In [14]:
conversion_times

,n_samples,yarrrml_to_rml_conversion_real_time,multiplier,n_run
0,704,8.881538,+25%,0
1,1000,8.438220,+25%,0
2,2000,8.805694,+25%,0
3,3000,8.397067,+25%,0
4,4000,8.789706,+25%,0
...,...,...,...,...
160,6000,8.009513,+100%,2
161,7000,8.579744,+100%,2
162,8000,8.556304,+100%,2
163,9000,8.485148,+100%,2


In [15]:
chart = alt.Chart(conversion_times).mark_line().encode(
    x=alt.X('n_samples', title="Number of materials libraries"),
    y=alt.Y('mean(yarrrml_to_rml_conversion_real_time)', title="YARRRML -> RML conversion time (seconds)",
            axis=alt.Axis(format=".1f")),
).properties(
    width=1920, height=1080 # For the paper: 800 x 600
)

line = chart.mark_line(size=3)
points = chart.mark_point(size=100)

text_1 = chart.mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=10
).encode(
    text=alt.condition(
        alt.datum.multiplier != "No multiplier",
        alt.Text('mean(yarrrml_to_rml_conversion_real_time):Q', format=".1f"),
        alt.value('')
    )
)

text_2 = chart.mark_text(
    align='left',
    baseline='middle',
    dx=5,
    dy=20
).encode(
    text=alt.condition(
        alt.datum.multiplier == "No multiplier",
        alt.Text('mean(yarrrml_to_rml_conversion_real_time):Q', format=".1f"),
        alt.value('')
    )
)

error_bands = alt.Chart(conversion_times).mark_errorband(extent='stdev').encode(
    x='n_samples:Q',
    y=alt.Y('mean(yarrrml_to_rml_conversion_real_time):Q', title='YARRRML -> RML conversion time (seconds)'),
)

chart = (chart + line + points + text_1 + text_2 + error_bands)

chart.save('avg_yarrrml_conversion_time.pdf')

# Main horizontal plot

## Total materialization time per number of samples (in multiple plots)

In [16]:
real_mat_times_charts = []

indexes_with_y_axis = [0]

for i, m in enumerate(['Original activity', '+25%', '+50%', '+75%', '+100%']):
    # This is horrible, but works (altair was refusing to share the Y axis...)
    if i in indexes_with_y_axis:
        chart_y_axis = alt.Y('mean(real_time)', title="Materialization time (seconds)", axis=alt.Axis(tickCount=10), scale=alt.Scale(type='log', base=2))
        error_y_axis = alt.Y('real_time:Q', title="Materialization time (seconds)")
    else:
        chart_y_axis = alt.Y('mean(real_time)', title="Materialization time (seconds)", 
                           axis=alt.Axis(labels=False, title=None, ticks=False, domain=False, tickCount=10), scale=alt.Scale(type='log', base=10))
        error_y_axis = alt.Y('real_time:Q', title="Materialization time (seconds)", 
                           axis=alt.Axis(labels=False, title=None, ticks=False, domain=False))
    
    base = alt.Chart(materialization_times.query("multiplier == @m")).encode(
        x=alt.X('n_samples', title="Number of materials libraries"),
        y=chart_y_axis,
    ).properties(
        title=f"Increased activity modifier: {m}" if i > 0 else "Original activity", 
        width=1920, # For the paper: 625
        height=1080 # For the paper: 550
    )
    
    error_layer = alt.Chart(materialization_times.query("multiplier == @m")).mark_errorband(extent='stdev', color='#001F3F').encode(
        x='n_samples',
        y=error_y_axis,
    )
    
    line_layer = base.mark_line(size=3, color='black') 
    point_layer = base.mark_point(size=100, filled=True, color='#001F3F')

    text_layer = base.encode(
        text=alt.Text('mean(real_time)', format='.1f'), 
    ).mark_text(
        align='center',
        baseline='middle',
        dx=0,
        dy=-15,
        color='#001F3F'
    ) 

    final_chart = error_layer + line_layer + point_layer + text_layer
    real_mat_times_charts.append(final_chart)

real_mat_times_chart = alt.hconcat(*real_mat_times_charts[:5], spacing=20).resolve_scale(
    y='shared'
)

## Per-mapping category materialization times

In [17]:
per_mapping_times_w_sums = (
    per_mapping_times.groupby(['category', 'n_samples', 'multiplier', 'n_run'], as_index=False) # Per run
      .agg(
          # Not representative, as all mappings are run in parallel
          real_time_sum_for_category=('mat_time', 'sum'), 
          
          # This corresponds to the maximum time for the mapping of each category, i.e.
          # the time for the last parallel job to finish for that category - hence, this
          # is the *real* time for the category
          real_time_max_for_category=('mat_time', 'max') 
      )
)

In [18]:
mat_times_per_cat_charts = []

indexes_with_y_axis = [0] 

for i, m in enumerate(['Original activity', '+25%', '+50%', '+75%', '+100%']):
    # Do the same as in the other materialization time plot
    if i in indexes_with_y_axis:
        bar_y_axis = alt.Y('mean(real_time_max_for_category):Q', title="Materialization time (seconds)", axis=alt.Axis(tickCount=10), scale=alt.Scale(type='log', base=2))
        error_y_axis = alt.Y('mean_time:Q', title="Materialization time (seconds)")
    else:
        y_axis_config = alt.Axis(labels=False, title=None, ticks=False, domain=False, tickCount=10)
        bar_y_axis = alt.Y('mean(real_time_max_for_category):Q', title="Materialization time (seconds)", axis=y_axis_config)
        error_y_axis = alt.Y('mean_time:Q', title="Materialization time (seconds)", axis=y_axis_config)
    
    bar_chart = alt.Chart(per_mapping_times_w_sums.query("multiplier == @m")).mark_bar().encode(
        x=alt.X('category:N', title="Mapping category", axis=alt.Axis(labelAngle=-45, labelLimit=0)),
        y=bar_y_axis,
        color=alt.Color('n_samples:N', 
                        legend=alt.Legend(title="Number of materials libraries", orient="bottom")),
        xOffset='n_samples:N'
    ).properties(title=f"Increased activity modifier: {m}" if i > 0 else "Original activity", width=1920, height=1080) # For the paper: 625 x 550
    
    error_chart = alt.Chart(per_mapping_times_w_sums.query("multiplier == @m")).transform_aggregate(
        mean_time='mean(real_time_max_for_category)',
        stdev_time='stdev(real_time_max_for_category)',
        groupby=['category', 'n_samples']
    ).mark_errorbar().encode(
        x=alt.X('category:N'), 
        y=error_y_axis,
        yError='stdev_time:Q',
        color=alt.Color('n_samples:N', legend=None),
        xOffset='n_samples:N'
    )
    
    mat_times_per_cat_charts.append(bar_chart + error_chart)

mat_times_per_category_chart = alt.hconcat(*mat_times_per_cat_charts, spacing=20).resolve_scale(y='shared')

In [19]:
top = alt.hconcat(*real_mat_times_charts, spacing=25).resolve_scale(x='shared', y='shared')
bot = alt.hconcat(*mat_times_per_cat_charts, spacing=70).resolve_scale(x='shared', y='shared')



#real_mat_times_chart.save('real_materialization_time_per_n_samples_per_multiplier_all.pdf')
#real_mat_times_chart


#mat_times_per_category_chart.save('materialization_time_per_category_and_n_samples_all.pdf')
#mat_times_per_category_chart

alt.vconcat(top, bot, spacing=50).save('materialization_time_per_category_and_n_samples_all_horizontal.pdf')

# Materialization CPU Usage

In [20]:
charts = [  
    alt.Chart(cpu_usage_trace.query("multiplier == @m")).mark_line().encode(
        x=alt.X('time:Q', title="Elapsed time (seconds)", axis=alt.Axis(labelAngle=0, tickCount=10)),
        y=alt.Y('mean(cpu_usage)', title="Total CPU usage (%)"),
        color=alt.Color(
            'n_samples:N',
            legend=alt.Legend(
                title="Number of materials libraries",
                orient="top"
            )
        ),
        shape=alt.Shape('multiplier', legend=None),
        strokeDash=alt.StrokeDash('multiplier', legend=None)
    ).properties(title=f"Multiplier: {m}", width=1920, height=1080)
    for m in ['Original activity', '+25%', '+50%', '+75%', '+100%']
]

top = alt.hconcat(*charts[:3], spacing=20)
bottom = alt.hconcat(
    alt.Chart().mark_text(text='').properties(width=125),
    *charts[3:],
    alt.Chart().mark_text(text='').properties(width=125) 
)

title = alt.TitleParams(
    text="Materialization CPU usage (% of total use over 24c/48t) (SQL database + RMLMapper and RMLStreamer mapping jobs) (Avg. across all tests)",
    offset=100,
    anchor='middle',
    fontSize=64
)
chart = alt.vconcat(top, bottom, spacing=40, title=title)

chart.save('materialization_cpu_usage.pdf')

# Materialization memory usage

In [21]:
per_mapping_memory_usage

,time,mapping_name,category,memory_usage,n_samples,multiplier,n_run
0,8,user ids rml materialized,,77070336,704,+25%,0
1,8,user given names rml materialized,,81002496,704,+25%,0
2,8,user names rml materialized,,81395712,704,+25%,0
3,8,projects rml materialized,,81002496,704,+25%,0
4,8,samples workflow instances and initial work rm...,,81002496,704,+25%,0
...,...,...,...,...,...,...,...
76089,47,measurements rml materialized,,2164535296,10000,+100%,2
76090,48,activities 11 rml materialized,,216809472,10000,+100%,2
76091,48,measurements rml materialized,,2289651712,10000,+100%,2
76092,49,activities 11 rml materialized,,316293120,10000,+100%,2


In [22]:
per_mapping_memory_usage['memory_usage_gib'] = per_mapping_memory_usage['memory_usage'] / (1024**3)

In [23]:
charts = [
    alt.Chart(per_mapping_memory_usage.query("multiplier == @m")).mark_line().encode(
    x=alt.X('time:Q', title="Elapsed time (seconds)", axis=alt.Axis(labelAngle=0, tickCount=10)),
    y=alt.Y('memory_usage_gib:Q', 
            title="Memory usage (GiB)", 
            aggregate='sum',
            axis=alt.Axis(format=".1f", tickCount=5)),
        color=alt.Color(
            'n_samples:N',
            legend=alt.Legend(
                title="Number of materials libraries",
                orient="top"
            )
        ),xOffset='n_samples:N'
    ).properties(title=f"Multiplier: {m}", width=1920, height=1080)
    for m in ['Original activity', '+25%', '+50%', '+75%', '+100%']
]

# 3 charts on the top, 2 charts centered on the bottom
top = alt.hconcat(*charts[:3], spacing=20)
bottom = alt.hconcat(
    alt.Chart().mark_text(text='').properties(width=125),  # spacer
    *charts[3:],
    alt.Chart().mark_text(text='').properties(width=125)   # spacer
)

title = alt.TitleParams(
    text="Materialization memory usage (SQL database + RMLMapper and RMLStreamer mapping jobs) (Avg. across all tests)",
    offset=100,
    anchor='middle',
    fontSize=64
)
chart = alt.vconcat(top, bottom, spacing=40, title=title)

chart.save('materialization_memory_usage_per_n_samples.pdf')

# Query benchmark results (w/ avgs)

In [24]:
query_benchmark_results_normal = query_benchmark_results[(query_benchmark_results["n_samples"] == 704) & (query_benchmark_results["multiplier"] == "Original activity")]
query_benchmark_results_extreme = query_benchmark_results[(query_benchmark_results["n_samples"] == 10000) & (query_benchmark_results["multiplier"] == "+100%")]

Avg across repetitions

In [25]:
query_benchmark_results_normal = query_benchmark_results_normal.groupby(['n_samples', 'multiplier', 'query', 'category', 'type'], as_index=False)['time'].mean()
query_benchmark_results_extreme = query_benchmark_results_extreme.groupby(['n_samples', 'multiplier', 'query', 'category', 'type'], as_index=False)['time'].mean()

In [26]:
def compute_winners(df):
    pivoted = df.pivot(index=['category', 'multiplier', 'query'], columns='type', values=['time']).reset_index()
    pivoted.columns = ['category', 'multiplier', 'query', 'time_sparql', 'time_sql']
    
    pivoted['sql_win'] = pivoted['time_sql'] < pivoted['time_sparql']
    pivoted['sparql_win'] = pivoted['time_sparql'] < pivoted['time_sql']
    
    sql_rows = pivoted[['category', 'multiplier', 'time_sql', 'sql_win']].rename(
        columns={'time_sql': 'time', 'sql_win': 'win'}
    )
    sql_rows['type'] = 'SQL'
    
    sparql_rows = pivoted[['category', 'multiplier', 'time_sparql', 'sparql_win']].rename(
        columns={'time_sparql': 'time', 'sparql_win': 'win'}
    )
    sparql_rows['type'] = 'SPARQL'
    
    df_with_winners = pd.concat([sql_rows, sparql_rows], ignore_index=True)
    return df_with_winners

query_benchmark_results_extreme = compute_winners(query_benchmark_results_extreme)
query_benchmark_results_normal = compute_winners(query_benchmark_results_normal)

x_label_order = ["Mat. libraries", "Measurements", "Compositions", "Handovers", "Users"]

In [27]:
query_benchmark_results_normal

,category,multiplier,time,win,type
0,Compositions,Original activity,0.089793,False,SQL
1,Compositions,Original activity,0.012660,True,SQL
2,Handovers,Original activity,0.041062,False,SQL
3,Handovers,Original activity,0.114237,False,SQL
4,Mat. libraries,Original activity,0.008519,True,SQL
5,Mat. libraries,Original activity,0.009095,False,SQL
6,Mat. libraries,Original activity,0.009726,True,SQL
7,Mat. libraries,Original activity,0.009961,False,SQL
8,Mat. libraries,Original activity,0.010555,True,SQL
9,Measurements,Original activity,0.005373,False,SQL


In [28]:
charts_avg = []
for df, title in [(query_benchmark_results_normal, "Original activity, 704 materials libraries"), (query_benchmark_results_extreme, "Increased activity (+100%), 10,000 materials libraries")]:
    base = alt.Chart(df).properties(width=600, height=550, title=title).encode(
        x=alt.X('category:N', title='Category', axis=alt.Axis(labelAngle=-45,  labelLimit=0), sort=x_label_order),
        color=alt.Color('type:N', scale=alt.Scale(domain=['SPARQL', 'SQL']), title='Datastore', legend=alt.Legend(title="Datastore", orient="top")),
        xOffset=alt.XOffset('type:N'),
        
    ).properties(width=1920, height=1080)

    # Compute mean and stdev per category/type
    agg = base.transform_aggregate(
        mean_time='mean(time)',
        stdev_time='stdev(time)',
        groupby=['category', 'type']
    ).transform_calculate(
        lower='max(0, datum.mean_time - datum.stdev_time)',
        upper='datum.mean_time + datum.stdev_time'
    )

    bars = agg.mark_bar(size=35).encode(
        y=alt.Y('mean_time:Q', title='Time (seconds)')
    )
    
    error_bars = agg.mark_errorbar(ticks=True, size=10, color='black').encode(
        y=alt.Y('lower:Q', title='Time (seconds)'),
        y2='upper:Q'
    )
    
    text = agg.mark_text(
        align='center',
        baseline='bottom',
        fontWeight='bold',
        dy=-2
    ).encode(
        y=alt.Y('mean_time:Q', title='Time (seconds)'),
        text=alt.Text('mean_time:Q', format='.2f')
    )


    charts_avg.append(bars + text + error_bars)

sparql_sql_benchmark_avgs = alt.hconcat(*charts_avg).resolve_scale(y='independent')
sparql_sql_benchmark_avgs.save('sparql_sql_benchmark_avg.pdf')

# Query benchmark results (w/o avgs)

In [29]:
query_benchmark_results_normal = query_benchmark_results[(query_benchmark_results["n_samples"] == 704) & (query_benchmark_results["multiplier"] == "Original activity")]
query_benchmark_results_extreme = query_benchmark_results[(query_benchmark_results["n_samples"] == 10000) & (query_benchmark_results["multiplier"] == "+100%")]

Avg. across repetitions

In [30]:
query_benchmark_results_normal = query_benchmark_results_normal.groupby(['n_samples', 'multiplier', 'query', 'category', 'type'], as_index=False)['time'].mean()
query_benchmark_results_extreme = query_benchmark_results_extreme.groupby(['n_samples', 'multiplier', 'query', 'category', 'type'], as_index=False)['time'].mean()

In [31]:
def compute_winners_no_avgs(df):
    pivoted = df.pivot(index=['category', 'multiplier', 'query'], columns='type', values=['time']).reset_index()
    pivoted.columns = ['category', 'multiplier', 'query', 'time_sparql', 'time_sql']
    
    pivoted['sql_win'] = pivoted['time_sql'] < pivoted['time_sparql']
    pivoted['sparql_win'] = pivoted['time_sparql'] < pivoted['time_sql']
    
    sql_rows = pivoted[['category', 'multiplier', 'query', 'time_sql', 'sql_win']].rename(
        columns={'time_sql': 'time', 'sql_win': 'win'}
    )
    sql_rows['type'] = 'SQL'
    
    sparql_rows = pivoted[['category', 'multiplier', 'query', 'time_sparql', 'sparql_win']].rename(
        columns={'time_sparql': 'time', 'sparql_win': 'win'}
    )
    sparql_rows['type'] = 'SPARQL'
    
    df_with_winners = pd.concat([sql_rows, sparql_rows], ignore_index=True)
    return df_with_winners

query_benchmark_results_extreme = compute_winners_no_avgs(query_benchmark_results_extreme)
query_benchmark_results_normal = compute_winners_no_avgs(query_benchmark_results_normal)

In [32]:
from textwrap import wrap

rename_map = {
    'samples_with_specific_elements': 'Materials libraries of silver or gold',
    'samples_with_specific_elements_exclusive': 'Materials libraries of silver and gold',
    'samples_and_authors': 'Materials libraries and creators',
    'samples_not_composed_of_elements': 'Materials libraries not of silver or gold',
    'samples_with_substrates': 'Substrate of every materials library',

    'get_edx_csv_measurements': 'All EDX measurements',
    'measurements_and_authors': 'Measurements and creators',
    
    'composition_count_per_MA': 'Measurement area compositions counts',
    'query_based_on_composition_values': 'Gold composition values',
    
    
    'handovers_with_specific_measurement_type': 'Handovers with XRD measurements',
    'measurement_to_handovers': 'Measurements to handover correspondences',
    
    'query_user_details': 'All user details',
    'samples_per_user': 'Materials libraries authored per user'
}

query_benchmark_results_normal['query'] = query_benchmark_results_normal['query'].replace(rename_map)
query_benchmark_results_extreme['query'] = query_benchmark_results_extreme['query'].replace(rename_map)

x_label_order = list(rename_map.values())

In [33]:
charts_no_avg = []
for df, title in [(query_benchmark_results_normal, "Original activity, 704 materials libraries"), (query_benchmark_results_extreme, "Increased activity (+100%), 10,000 materials libraries")]:
    base = alt.Chart(df).properties(width=600, height=550, title=title).encode(
        x=alt.X('query:N', title='Query', axis=alt.Axis(labelAngle=-45, labelLimit=0, titlePadding=125), sort=x_label_order),
        y=alt.Y('time:Q', title='Time (seconds)'),
        color=alt.Color('type:N', scale=alt.Scale(domain=['SPARQL', 'SQL']), title='Datastore'),
        xOffset=alt.XOffset('type:N')
    ).mark_bar(size=15).properties(width=1920, height=1080)
    
    text = base.mark_text(
        align='center',
        baseline='bottom',
        fontSize=12,
        fontWeight='bold',
        dy=-2
    ).encode(
        y=alt.Y('time:Q', title='Time (seconds)'),
        text=alt.Text('time:Q', format='.3f')
    )

    charts_no_avg.append(base + text)

sparql_sql_benchmark_no_avgs = alt.hconcat(*charts_no_avg).resolve_scale(y='independent')
sparql_sql_benchmark_no_avgs.save('sparql_sql_benchmark_no_avg.pdf')

# Second horizontal plot

In [34]:
second_plot = alt.hconcat(
    n_triples_plot,
    postprocessing_plot,
    sparql_sql_benchmark_avgs
).resolve_scale(
    x='independent', 
    y='independent',  
    color='independent',
    #detail='independent'
).resolve_axis(
    x='independent',
    y='independent'
)

second_plot.save('second_horizontal_plot.pdf')

# Query benchmark stats

## General

In [35]:
query_benchmark_results[query_benchmark_results["type"] == "sql"]["time"].mean()

np.float64(3.7821464586975857)

In [36]:
query_benchmark_results[query_benchmark_results["type"] == "sql"]["time"].std()

np.float64(18.057030083382685)

In [37]:
query_benchmark_results[query_benchmark_results["type"] == "sparql"]["time"].mean()

np.float64(0.09348216584562336)

In [38]:
query_benchmark_results[query_benchmark_results["type"] == "sparql"]["time"].std()

np.float64(0.2534307887870314)

## Samples, measurements and users

In [39]:
def get_difference(sql_times, sparql_times):
    n_sql = len(sql_times)
    n_sparql = len(sparql_times)

    mean_sql = sql_times.mean()        
    mean_sparql = sparql_times.mean()  
    std_sql = sql_times.std()           
    std_sparql = sparql_times.std() 
    
    sem_sql = std_sql / np.sqrt(n_sql)
    sem_sparql = std_sparql / np.sqrt(n_sparql)
        
    diff_mean = mean_sql - mean_sparql
    
    diff_sem = np.sqrt(sem_sql**2 + sem_sparql**2)
    
    print(f"Difference in means: {diff_mean}")
    print(f"Propagated Standard Error: {diff_sem}")
    print(f"SEM: {diff_mean:.3f} +- {diff_sem:.3f}")

Avg. difference in the real distribution of data

In [40]:
sql = query_benchmark_results_normal[(query_benchmark_results_normal["type"] == "SQL")]["time"]
sparql = query_benchmark_results_normal[(query_benchmark_results_normal["type"] == "SPARQL")]["time"]

get_difference(sql, sparql)

Difference in means: 0.004416152094610229
Propagated Standard Error: 0.012061625879666794
SEM: 0.004 +- 0.012


Avg. difference in the largest db for handovers and compositions

In [41]:
extreme_handovers_sql = query_benchmark_results_extreme[
    (query_benchmark_results_extreme["type"] == "SQL") & 
    (query_benchmark_results_extreme["category"].isin(["Handovers"]))
]["time"]

extreme_handovers_sparql = query_benchmark_results_extreme[
    (query_benchmark_results_extreme["type"] == "SPARQL") & 
    (query_benchmark_results_extreme["category"].isin(["Handovers"]))
]["time"]

get_difference(extreme_handovers_sql, extreme_handovers_sparql)

Difference in means: 9.984077991684899
Propagated Standard Error: 0.05797578768073231
SEM: 9.984 +- 0.058


In [42]:
extreme_handovers_sql = query_benchmark_results_extreme[
    (query_benchmark_results_extreme["type"] == "SQL") & 
    (query_benchmark_results_extreme["category"].isin(["Compositions"]))
]["time"]

extreme_handovers_sparql = query_benchmark_results_extreme[
    (query_benchmark_results_extreme["type"] == "SPARQL") & 
    (query_benchmark_results_extreme["category"].isin(["Compositions"]))
]["time"]

get_difference(extreme_handovers_sql, extreme_handovers_sparql)

Difference in means: 1.7260581427641835
Propagated Standard Error: 3.406442077455285
SEM: 1.726 +- 3.406


# Materialization statistics

In [43]:
materialization_times

,n_samples,real_time,multiplier,n_run
0,704,55.351614,+25%,0
1,1000,57.278403,+25%,0
2,2000,127.720173,+25%,0
3,3000,176.900779,+25%,0
4,4000,145.072490,+25%,0
...,...,...,...,...
160,6000,1694.376703,+100%,2
161,7000,2203.878119,+100%,2
162,8000,2200.648775,+100%,2
163,9000,2524.543448,+100%,2


In [44]:
print(materialization_times[
    (materialization_times["multiplier"] == "Original activity") 
]["real_time"].std() / 60.0)

0.7625575514650114


In [45]:
print(materialization_times[
    (materialization_times["multiplier"] == "+25%") 
]["real_time"].std() / 60.0)

3.3023100340986438


In [46]:
print(materialization_times[
    (materialization_times["multiplier"] == "+50%") 
]["real_time"].std() / 60.0)

4.269579348826125


In [47]:
print(materialization_times[
    (materialization_times["multiplier"] == "+75%") 
]["real_time"].std() / 60.0)

8.752195167727468


In [48]:
print(materialization_times[
    (materialization_times["multiplier"] == "+100%") 
]["real_time"].std() / 60.0)

20.22548067716234
